In [96]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, import_plt_file #these python classes should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, import_plt_file]:
    reload(obj)

import numpy as np

In [97]:
search_for_word = 'source'
search_inside_chapter = 'Radiation'
df = search.Generators().word(input = search_for_word, word_slicing = (0,10), print_path = True)
print(df.to_string())
print(search.Generators().chapter(input = search_inside_chapter))


/home/brewster/Desktop/cretin.v2_19_test/test//serial9/serial9.gen
/home/brewster/Desktop/cretin.v2_19_test/test//serial1/serial1.gen
/home/brewster/Desktop/cretin.v2_19_test/test//trial0/trial0.gen
/home/brewster/Desktop/cretin.v2_19_test/test//trial0/trial0.gen
/home/brewster/Desktop/cretin.v2_19_test/test//trial0/trial0.gen
/home/brewster/Desktop/cretin.v2_19_test/test//scat2d/scat2d.gen
/home/brewster/Desktop/cretin.v2_19_test/test//lym3d/lym3d.gen
/home/brewster/Desktop/cretin.v2_19_test/test//serial7/serial7.gen
/home/brewster/Desktop/cretin.v2_19_test/test//advect/advect.gen
/home/brewster/Desktop/cretin.v2_19_test/test//advect/advect.gen
/home/brewster/Desktop/cretin.v2_19_test/test//advect/advect.gen
/home/brewster/Desktop/cretin.v2_19_test/test//advect/advect.gen
/home/brewster/Desktop/cretin.v2_19_test/test//serial5/serial5.gen
/home/brewster/Desktop/cretin.v2_19_test/test//trial8/trial8.gen
/home/brewster/Desktop/cretin.v2_19_test/test//trial8/trial8.gen
/home/brewster/Desk

In [98]:
# first we create our generic object
def sim(var1, var2, var3):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9, 3, 512,  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 5 # the laser energies
    DN, DR = 10, .1


    gen2.materials_atom(index = 1, quantum_n_max = 5, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-7, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 3)
    gen2.source_laser(laser_wavelength = var2, option_1= 'rate', option_2= 'history', laser_intensities = [1,1], nodes= [N0, N1])
    gen2.source_jnu(E_range=[E1,var3], option_1= 'rate', option_2= 'history', nodes= [N0, var1])
    gen2.source_jbndry(index = 1, E_range=[E1,var2], option_1= 'rate', option_2= 'history', nodes = [1,var2], values=[1,var2])

    return gen2


i0, iend = 0,10
trials = []
name = 'trial'
for i in range(i0,iend):

    trial = name+str(i)
    trials.append(trial)
    var1 = 3
    var2 = 4**i
    var3 = 1.3
    gen2 = sim(var1, var2, var3)
    write_run_plot.write(name = name+str(i), object = gen2)
    write_run_plot.run(name = name+str(i), longprint=False)

serial_sim_tools.plot_all('trials_mutiplot', trials)


writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial0/trial0.gen
running cretin with trial0

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial1/trial1.gen
running cretin with trial1

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial2/trial2.gen
running cretin with trial2

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial3/trial3.gen
running cretin with trial3

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial4/trial4.gen
running cretin with trial4

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial5/trial5.gen
running cretin with trial5

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial6/trial6.gen
running cretin with trial6

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial7/trial7.gen
running cretin with trial7

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial8/trial8.gen
running cretin with trial8

writing too /home/brewster/Desktop/cretin.v2_19_test/test/trial